IzLasgdLEg- open api 필요 (Get)
  - 관련사이트
    - dev.naver.com
    - dev.kakao.com
  - Client_ID (관리)
    - hgRZ5Mwe5tQwFPqyl_hg
  - Client Secret (관리)
    - IzLasgdLEg
  - API 문서
    - https://developers.naver.com/docs/search/news/
  - 검색 URL (응답 데이터 json)
    - https://openapi.naver.com/v1/search/news.json

  - 예시

```
curl "https://openapi.naver.com/v1/search/news.xml?query=%EC%A3%BC%EC%8B%9D&display=10&start=1&sort=sim" \
    -H "X-Naver-Client-Id: {애플리케이션 등록 시 발급받은 client id 값}" \
    -H "X-Naver-Client-Secret: {애플리케이션 등록 시 발급받은 client secret 값}" -v
```




- request 모듈 필요->통신수행(http)
  - get방식을 주로 사용
  - 개인별 인증키는 헤더에 숨겨서 전송

## 구현

In [1]:
# 1. 필요한 모듈 가져오기
import urllib.request
import os
import sys

In [2]:
# 2. 환경변수, 통신에 필요한 키를 정의
Client_ID     = 'pP7lQhkZAx70dyk3sLcJ'
Client_Secret = 'twyoc8OdDG'

In [3]:
# 3. URL 정의
naver_news_url= 'https://openapi.naver.com/v1/search/news.json'
naver_news_url

'https://openapi.naver.com/v1/search/news.json'

In [5]:
# 4. 파라미터 정의
# 한글이 검색어인 경우, utf-8 인코딩 처리 해야한다. 
# 한글 => %EA%AA..
keyword = urllib.parse.quote('코로나')
keyword
param   = 'query=' + keyword
param

'query=%EC%BD%94%EB%A1%9C%EB%82%98'

In [6]:
# 5. 최종 get방식으로 요청하는 url 
req_url = f'{naver_news_url}?{param}'
req_url

'https://openapi.naver.com/v1/search/news.json?query=%EC%BD%94%EB%A1%9C%EB%82%98'

In [7]:
# 6. 통신 객체 생성
# from urllib.request import Request
request = urllib.request.Request( req_url )

In [8]:
# 7. 클라이언트 키, 보안 키 헤더에 등록
'''
-H "X-Naver-Client-Id: {애플리케이션 등록 시 발급받은 client id 값}"
-H "X-Naver-Client-Secret: {애플리케이션 등록 시 발급받은 client secret 값}" -v
'''
request.add_header( 'X-Naver-Client-Id',     Client_ID     )
request.add_header( 'X-Naver-Client-Secret', Client_Secret )

In [9]:
# 8. 통신
response = urllib.request.urlopen( request )

In [10]:
# 9. 응답코드 확인 -> 200:http로 통신하고나서 오는 응답코드중 정상을 의미
# 401:권한오류
# 404:해당페이지가 없다 
# 405:해당권한이없다 
# 500:서버내부에러
response.getcode()

200

In [12]:
# 10. 응답 데이터를 json 객체로 로드하겟다 => 딕녀서리, 리스트 조합 구성
import json

if response.getcode() == 200:
  res = json.load( response )
else:
  print('error', response.getcode())

In [15]:
# 11. 파싱 
# 응답된 데이터에서 description 항목을 모두 출력하시오
len( res['items'] )

10

In [18]:
for news in res['items']:
  #print( news['description'] )
  print( news['title'].replace('<b>','').replace('</b>','') )
  # 데이터들을 좀더 가공해서 데이터베이스에 적재하겠다 <- 전처리(정규식, 기타..)

산은, ‘사랑의 김장 나누기’ 행사 열어
양주시 소재 새순교회, 사랑의 백미 600㎏ 기부
이재명 지사, 정세균 총리와 경기도 홈케어시스템 등 코로나19 대응책 논의
김포 우리들병원, 해외 의료진 대상 '척추치료' 선진의술 전수
부천시, 동절기 대비 건설공사장 안전 점검
서울시, 문화누리카드 잔액 인증 이벤트 '0말정산' 진행
경북 구미 일가족 코로나19 확진 판정
&quot;주제넘은 평가, 대가 치를 것&quot; 北 김여정 한 마디에… 文 강경화 해임설
춘천문화재단, '나의 도시, 더 나은 삶의 이야기' 개최
건보공단서 의료기관 개설시 예타 면제 추진


## 데이터 적제

- 파일
  - csv, xls(x)
  - json
- 데이터베이스
  - RDBMS
    - 기업형 디비, ms-sql, oracle,..    
    - 현재 작업 기준에서는 코랩에서 작동시, **고정 IP 나 도메인을 가진 데이터베이스**를 활용하여 저장해야한다
      - aws 사용
  - No-Sql
    - 몽고디비 -> 로그 저장

- 절차
  - 데이터 구조 : [ {},{},{},... ] 준비
  - pandas를 이용하여 DataFrame 구성
  - pymysql + sqlalchemy 이용하여 접속
  - 데이터를 데이터베이스에 적제
  - 연결 종료

In [19]:
# 1. 수집한 데이터를 DataFrame으로 일단 변형
import pandas as pd

In [20]:
df = pd.DataFrame.from_dict( res['items'] )
df.head(2)

,title,originallink,link,description,pubDate
0,"산은, ‘사랑의 김장 나누기’ 행사 열어",http://www.pinpointnews.co.kr/view.php?ud=2020...,http://www.pinpointnews.co.kr/view.php?ud=2020...,올해는 <b>코로나</b>19 확산방지를 위해 사회적기업으로부터 김장김치를 구매하여...,"Wed, 09 Dec 2020 14:40:00 +0900"
1,"양주시 소재 새순교회, 사랑의 백미 600㎏ 기부",http://www.straightnews.co.kr/news/articleView...,http://www.straightnews.co.kr/news/articleView...,최영완 담임목사는 &quot;<b>코로나</b>19 여파로 힘든 시기를 보내고 있는...,"Wed, 09 Dec 2020 14:40:00 +0900"


## 단일 모듈으로 덤프

- xxx.py 저장

## 자동화

- OS 레벨에서 정해진 스케쥴에서 의해서 자동으로 수집하겠금 구성